# Pendulo invertido
En este archivo se puede observar como es el entrenamiento de un modelo de Deep Reinforcement Basico, entrenado con un algoritmo PPO y sin modificar sus parámetros por defecto.

In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'


import gymnasium as gym
import numpy as np
import os
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

En la siguiente dirección se guardan (o cargan) los modelos a utilizar.

In [3]:
path = "models/cartpole"

Para este caso, solo se modifican los siguientes parámetros. `num_cpu` y `test_num_cpu` se puede modificar dado que se ha creado un entorno vectorizado, una herramienta muy útil para aprovechar el multiprocesamiento de la CPU o de la GPU.

In [4]:
num_cpu = 6  # Numero de nucleos a utilizar para el entrenamiento
test_num_cpu = 1 # Numero de nucleos a utilizar para la evaluacion. Dejar en 1 para evitar errores
total_timesteps = 250000 
env_id = "CartPole-v1" 

Se crean los entornos, así como un objeto para hacer un seguimiento del entrenamiento y poder guardar el mejor modelo obtenido, que no necesariamente es al final del entrenamiento.

In [5]:
vec_env = make_vec_env(env_id, n_envs=num_cpu)

eval_env = make_vec_env(env_id, n_envs=1)

eval_callback = EvalCallback(eval_env, best_model_save_path=path,
                                log_path=path, eval_freq=30000 // num_cpu,
                                n_eval_episodes=7, deterministic=True,
                                render=False)

Creación del modelo

In [6]:
model = PPO("MlpPolicy", vec_env, verbose=0, tensorboard_log=path)

El entrenamiento se realiza a continuación. Los outputs se encuentran colapsados para evitar ocupar demasiado espacio, pero se pueden descolapsar para ver cuál ha sido el procedimiento.

**No es necesario entrenar de nuevo el modelo, ya que se encuentra guardado en la carpeta `models`**

In [ ]:
model.learn(total_timesteps=total_timesteps, progress_bar=True, callback=eval_callback, tb_log_name="TB_LOG")

del model # Se borra para que luego se pueda cargar el modelo sin problemas

Para poder probar el modelo, aunque se podria usar el entorno `vec_env`, vamos a crear un entorno individual para evitar errores (a veces Stable Baselines 3 y PyTorch dan problemas de compatibilidad, y así se ahorran)

Se carga el modelo de la carpeta `models`, que corresponde al entrenado anteriormente

In [7]:
model = PPO.load("models/cartpole/best_model", env=eval_env)

La función `evaluate_policy` permite examinar el modelo en el escenario, proporcionando una media de recompensas obtenidas y su desviación típica. Dado el rendimiento del algoritmo PPO y la simpleza del caso, consigue una precisión perfecta en todas las circunstancias (o, al menos, en casi todas).

In [8]:
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=40)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:500.00 +/- 0.00


A continuación, se puede visualizar una secuenciad e iteraciones del modelo. Como se visualizó en la evaluación del modelo, la precisión será perfecta.

***Nota: En ocasiones, ipynb da problemas para visualizar la ejecución. En ese caso, utilizar el codigo en la carpeta `otros`.***

In [10]:
obs = eval_env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = eval_env.step(action)
    eval_env.render("human")